# BinaPs Demo

Copyright 2022 Bernardo C. Rodrigues

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public
License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later
version. This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the
implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more
details. You should have received a copy of the GNU General Public License along with this program. If not, 
see <https://www.gnu.org/licenses/>. 

This notebook demonstrates the usability of the BinaPs implementation provided by its authors. It consists on three
steps:

**1. Generate synthetic data:** With one of the provided scripts, we generate synthetic data in which a set of known
patterns are planted.

**2. Run BinaPs:** We'll run BinaPs over this synthetic dataset and get a list of inferred patterns

**3. Compare inferred to original patterns**: We use another provided script to calculate the 
[F1](https://en.wikipedia.org/wiki/F-score) value between the inferred and original patterns.

In [1]:
import torch

# Check if CUDA is available
if(torch.cuda.is_available()):
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
else:
    print("No CUDA device avaiable")

CUDA Device: NVIDIA GeForce GTX 1050


## 1. Generate synthetic data

In this step we will end up with 4 files:

```
...
├── data.dat
├── data.dat_patterns.txt
├── data_itemOverlap.dat
├── data_itemOverlap.dat_patterns.txt
...
```

*data.dat* and *data_itemOverlap.dat* are the synthetic datasets while *data.dat_patterns.txt* are their respective
*data_itemOverlap.dat_patterns.txt* root patterns respectively. Arbitrary noise is added to test the BinaPS robustness.

In *data_itemOverlap.dat* the patterns may overlap (e.g ABC CDE CEF) while in *data.dat* they may not (e.g. AB C DE F).


In [2]:
from lib.utils import generate_synthetic_data

output_file = f"data"

row_quantity = 10000
column_quantity = 50
max_pattern_size = 10
noise = 0.001
density = 0.05

generate_synthetic_data(row_quantity, column_quantity, output_file, max_pattern_size, noise, density)

Rscript binaps/Data/Synthetic_data/generate_toy.R AND 50 10000 10 data 0.001 0.05
[1] 291
[1] 10000   291
[1] "Added noise."
[1] "Converted to dat file."
[1] "Removed rows without content."
         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 380981 20.4     809590 43.3   641325 34.3
Vcells 874268  6.7    8388608 64.0  8388589 64.0
[1] 10000   185



## 2. Run BinaPs

A complete run of th BinaPs autoencoder. In this step we'll end up with the inferred patterns:
```
...
├── data.binaps.patterns
...
```

Attention, patterns in this output start at '0', as opposed to patterns generated by
*generate_synthetic_data* that start at '1'. In other words, the pattern {0,1,5} is equal to {1,2,6}
in the original dataset.


In [3]:
from lib.utils import run_binaps

data_path = "data.dat"
hidden_dimension = 20
epochs = 100

run_binaps(data_path, hidden_dimension, epochs)

python3 binaps/Binaps_code/main.py -i data.dat --hidden_dim 20 --epochs=100
Data Sparsity:
0.05291526162004616
Train Epoch: 1 [0/8464 (0%)]	Loss: 12.980500
Train Epoch: 1 [640/8464 (8%)]	Loss: 11.461599
Train Epoch: 1 [1280/8464 (15%)]	Loss: 10.641851
Train Epoch: 1 [1920/8464 (23%)]	Loss: 11.389548
Train Epoch: 1 [2560/8464 (30%)]	Loss: 9.996649
Train Epoch: 1 [3200/8464 ( [output truncated]
...
[output truncated]  83  84  85 116 117 118 119 120 121 122 285 286 287 288 289 290] ( 4 / 78 )
[ 54 142 143 144 145 146 199 200 201 202 203 204 205 206 207 208] ( 2 / 410 )
[ 60  61  62  63  64  65  66  67  68  69 209 210 211 212 213] ( 32 / 457 )
[183 184 185 186 187 188 189 190 191 192 195 196 197 198] ( 28 / 497 )



## 3. Compare inferred to original patterns

Get the F1 score for the inferred patterns.

compare_datasets_based_on_f1 takes into account the diference of the patterns' start mentioned at
the previous step.

In [4]:
from lib.utils import compare_datasets_based_on_f1

estimated_patterns_file = "data.binaps.patterns"
real_patterns_file = "data.dat_patterns.txt"

compare_datasets_based_on_f1(estimated_patterns_file, real_patterns_file)

0.1095890410958904



# Overlapped patterns example

In [5]:
data_path = "data_itemOverlap.dat"
hidden_dimension = 20
epochs = 100

run_binaps(data_path, hidden_dimension, epochs)

python3 binaps/Binaps_code/main.py -i data_itemOverlap.dat --hidden_dim 20 --epochs=100
Data Sparsity:
0.08227798815783022
Train Epoch: 1 [0/8434 (0%)]	Loss: 12.642015
Train Epoch: 1 [640/8434 (8%)]	Loss: 12.455598
Train Epoch: 1 [1280/8434 (15%)]	Loss: 8.444002
Train Epoch: 1 [1920/8434 (23%)]	Loss: 10.501066
Train Epoch: 1 [2560/8434 (30%)]	Loss: 9.911861
Train Epoch: 1 [3200/8434 (3 [output truncated]
...
[output truncated] 37  39  50  53  58  61  64  93  95 118 126 164 168 170] ( 0 / 202 )
[  6  22  28  38  68  72  77  79 112 120 121 132 158 159 160 172] ( 4 / 187 )
[  7  18  24  34  42  47  51  54  59  74  78 135 139 143 149 166 173] ( 20 / 554 )
[  5  53  62  85  94  98 102 105 112 114 145 150 178 181] ( 28 / 498 )



In [6]:
estimated_patterns_file = "data_itemOverlap.binaps.patterns"
real_patterns_file = "data_itemOverlap.dat_patterns.txt"

compare_datasets_based_on_f1(estimated_patterns_file, real_patterns_file)

0.28169014084507044

